In [ ]:
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd

In [ ]:
# Set up the Selenium Web Driver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
wait = WebDriverWait(driver, 10)

# Open the initial page
driver.get("https://webgate.ec.europa.eu/rasff-window/screen/search")

# Find the element of the button and click on the button
try:
    button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "button[type='submit']")))
    button.click()
except:
    driver.quit()

# Empty lists to store scraped data
categories = []
Types = []
subjects = []
dates = []
Classes = []
decisions = []

def extract_data():
    # Extract Category
    categories_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "item-product-category")))
    for category_element in categories_elements:
        categories.append(category_element.text)

    # Extract Type
    types_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "item-product-type")))
    for type_element in types_elements:
        Types.append(type_element.text)

    # Extract Subjects
    subjects_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "item-subject")))
    for subject_element in subjects_elements:
        subjects.append(subject_element.text)

    # Extract Date
    dates_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "item-validation-date")))
    for date_element in dates_elements:
        dates.append(date_element.text)

    # Extract Notifying Country
    #countries_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "item-notification-classification")))
    #for country_element in countries_elements:
        #origins.append(country_element.text)

    # Extract Class
    classes_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "item-notification-classification")))
    for class_element in classes_elements:
        Classes.append(class_element.text)

    # Extract Decision
    decisions_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "item-risk-decision")))
    for decision_element in decisions_elements:
        decisions.append(decision_element.text)

# Extract data from current page
extract_data()

# Perform actions on each page
page_num = 1
while True:
    try:
        # Wait for the results to load
        div = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.results-list-body")))
        print(f"Results on page {page_num}:\n{div.text}\n")

        # Extract data from current page
        extract_data()

        # Check if there is a next page
        next_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "button.mat-paginator-navigation-next")))
        if "disabled" in next_button.get_attribute("class"):
            break  # Exit the loop if there is no next page

        # Perform a click action using ActionChains
        action = ActionChains(driver)
        action.move_to_element(next_button).click().perform()
        page_num += 1

        # Add a delay to allow the next page to load
        time.sleep(2)
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        break

# Create a DataFrame from the scraped data
data = {
    "Category": categories,
    "Product": Types,
    "Subject": subjects,
    "Date": dates,
    "Class": Classes,
    "Decision": decisions
}
df = pd.DataFrame(data)
df